In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative, ignore_index=True)

In [4]:
def preprocess_df(df):
    
    df.text = df.text.apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
    
    for index in range(0, len(df)):
        result = ''.join((x for x in df[df.index == index].text.values[0] if not x.isdigit()))
        result = re.sub("_", " ", result)
        result = re.sub(r'\s*[A-Za-z]+\b', '' , result)
        result = re.sub("\n"," ",result)
        df.loc[df.index == index, 'text'] = result
    
    return df

In [5]:
morph = pymorphy2.MorphAnalyzer()
words_regex = re.compile('\w+')
stopwords_list = stopwords.words('russian')

In [6]:
def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [7]:
preprocessed_text = list(tqdm(map(preprocess, df['text']), total=len(df)))

  0%|          | 0/226834 [00:00<?, ?it/s]

In [8]:
df['text'] = preprocessed_text
df.sample(3)

,text,label
119205,"[передумать, хотеть, смотреть, проблема]",negative
115686,"[графоманить, сень, хотеться, стёб, писать, бо...",negative
136695,"[satanabitch, саша, уебал, кожицой, мандарин, ...",negative


In [9]:
from gensim.models import *
from gensim import corpora

In [10]:
dictionary = corpora.Dictionary(df['text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('df.dict')

In [11]:
corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('df.model', corpus) 

In [12]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10, chunksize=50, update_every=1, passes=2)

CPU times: total: 2min 52s
Wall time: 3min 1s


In [13]:
lda.show_topics(num_topics=10, num_words=10, formatted=True)

[(0,
  '0.149*"http" + 0.113*"хотеть" + 0.044*"знать" + 0.033*"делать" + 0.030*"спать" + 0.024*"скучать" + 0.016*"бля" + 0.016*"голова" + 0.014*"пока" + 0.012*"выходной"'),
 (1,
  '0.056*"блин" + 0.050*"всё" + 0.043*"свой" + 0.038*"писать" + 0.033*"который" + 0.032*"время" + 0.028*"хотеться" + 0.022*"каждый" + 0.020*"работа" + 0.018*"вчера"'),
 (2,
  '0.120*"день" + 0.060*"любить" + 0.046*"пойти" + 0.037*"мама" + 0.031*"твой" + 0.031*"неделя" + 0.027*"настроение" + 0.027*"скоро" + 0.024*"второй" + 0.017*"папа"'),
 (3,
  '0.078*"очень" + 0.056*"человек" + 0.044*"сказать" + 0.038*"хороший" + 0.031*"говорить" + 0.020*"холодно" + 0.020*"дело" + 0.017*"телефон" + 0.016*"хотя" + 0.014*"ранний"'),
 (4,
  '0.151*"это" + 0.069*"мочь" + 0.059*"сегодня" + 0.026*"болеть" + 0.021*"час" + 0.017*"друг" + 0.016*"ждать" + 0.015*"любимый" + 0.014*"посмотреть" + 0.013*"нужно"'),
 (5,
  '0.114*"ещё" + 0.061*"почему" + 0.047*"большой" + 0.027*"утро" + 0.027*"ночь" + 0.020*"нибыть" + 0.019*"остаться" + 0.01

In [14]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

lda_viz = gensimvis.prepare(lda, corpus, dictionary,  mds='mmds')

D:\Soft\conda\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [15]:
pyLDAvis.display(lda_viz)

проинтерпритировать получившиеся тематики <br>
Как и было рассказано на занятии, для более точного решения данных задач нужно скрупулёзно обрабатывать данные <br>
Из 10ти тематик могу примерно выделить следующие: <br>
    1. Болезнь<br>
    2. Выходной<br>
    4. Работа<br>
    5. Школа/учеба<br>
    

